In [ ]:
from Utils.my_utils import *
from Recommenders.SLIM.SLIMElasticNetRecommender import *
import numpy as np
from Evaluation.Evaluator import EvaluatorHoldout
import os
import optuna
import pandas as pd

In [ ]:
users, items, users_to_eval = readData()
URM = newPreProcess()
URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM, train_percentage = 0.80)
evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])

In [ ]:
def objective_function_SLIM(optuna_trial):
    recommender_instance = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)
    recommender_instance.fit(l1_ratio = optuna_trial.suggest_float("l1_ratio", 1e-4, 0.1, log=True),
                             alpha = optuna_trial.suggest_float("alpha", 1e-3, 0.01),
                             topK = optuna_trial.suggest_int("topK", 1, 999))
        
    result_df, _ = evaluator_validation.evaluateRecommender(recommender_instance)
    
    return result_df.loc[10]["MAP"]

In [ ]:
optuna_study = optuna.create_study(direction="maximize")
        
save_results = SaveResults()
        
optuna_study.optimize(objective_function_SLIM,
                      callbacks=[save_results],
                      n_trials = 20)

In [ ]:
idx = save_results.results_df['result'].idxmax()
best_result = save_results.results_df.loc[idx]["result"]
print("MAP: " + str(best_result))
with open('best_parameters.txt', 'w') as file:
    file.write(optuna_study.best_trial.params)

In [ ]:
# Best parameters found
optuna_study.best_trial.params

In [ ]:
# Fit with best parameters found
recommender_instance = MultiThreadSLIM_SLIMElasticNetRecommender(URM)
recommender_instance.fit(**optuna_study.best_trial.params)

In [ ]:
# Recommend items to users
recommendations = recommender_instance.recommend(users_to_eval, cutoff=10)

# Store recommendations
newStoreRecommendations(users_to_eval, recommendations)